# Higher-order Tensor Renormalization Group for 2D Ising model

In [1]:
import numpy as np
from numpy.typing import ArrayLike
import hotrg
import ipywidgets as widgets
import matplotlib.pyplot as plt
%config InlineBackend.figure_formats = ["svg"]
import itertools
from tqdm.auto import tqdm
import cProfile
import opt_einsum as oe

The average **magnetization** $\braket{m}$ of a statistical system can be computed as,

$$ \langle\text{Magnetization}\rangle = \frac{\text{Tr}(H)}{Z} $$

The **partition function** $Z$ can be described as a translation-invariant tensor network contraction.

$$ Z = \sum_{i} e^{- \beta E_i} = \text{Tr} \prod_i T_{l_i r_i u_i d_i}(\beta) $$

where

$$
W = \begin{pmatrix}
\sqrt{\cosh(1/t)} & \sqrt{\sinh(1/t)} \\
\sqrt{\cosh(1/t)} & -\sqrt{\sinh(1/t)}
\end{pmatrix}
$$

$$
T_{lrud} = \sum_\alpha W_{\alpha,l} W_{\alpha,r} W_{\alpha,u} W_{\alpha,d}
$$

The $H$ tensor has a similar definition to $T$.
$$ H^{(0)}_{lrud} = \sum_\alpha W_{\alpha,l} ~ W_{\alpha,r} ~ W_{\alpha,u} ~ W_{\alpha,d} ~ m_\alpha ~~~~ \text{where} ~~~~ m_\alpha = \begin{pmatrix} 1 \\ -1 \end{pmatrix} $$

## Procedure

1. Contract $T$ with the tensor below and reshape to a 4-order tensor $M$.

$$ M^{(n)}_{lrud} = \sum_\alpha T^{(n-1)}_{l_1 r_1 u \alpha} T^{(n-1)}_{l_2 r_2 \alpha d} ~~~~ \text{where} ~~~~ l = l_1 \otimes l_2, ~~ r = r_1 \otimes r_2 $$

In [2]:
def contract_tensors(top: np.ndarray, bottom: np.ndarray, backend="numpy") -> np.ndarray:
	m = oe.contract("abci,deif->adbecf", top, bottom, backend=backend)
	return m.reshape((m.shape[0]*m.shape[1], m.shape[2]*m.shape[3], m.shape[4], m.shape[5]))


2. Perform the HOSVD on $M$.

$$ M^{(n)}_{lrud} = \sum_{\alpha \beta \gamma \epsilon} S_{\alpha \beta \gamma \epsilon} U^L_{l,\alpha} U^R_{r,\beta} U^U_{u,\gamma} U^D_{d,\epsilon} $$

In [3]:
def hosvd_sides(A: ArrayLike) -> tuple[ArrayLike, list[ArrayLike]]:
    """Higher-order singular value decomposition (HOSVD)

    parameters
    ----------
    - A. ArrayLike

    returns
    -------
    - S. ArrayLike
    - U. Sequence[ArrayLike]
    """

    # left SVD
    Ak = A.reshape((A.shape[0], -1))
    Ul, S, Vh = np.linalg.svd(Ak, full_matrices=False, compute_uv=True, hermitian=False)
    
    Ak = np.atleast_2d(S).T * Vh
    Ak = Ak.reshape(A.shape)

    # right SVD
    Ak = np.moveaxis(Ak, 1, 0).reshape((A.shape[1], -1))
    Ur, S, Vh = np.linalg.svd(Ak, full_matrices=False, compute_uv=True, hermitian=False)

    Ak = np.atleast_2d(S).T * Vh
    Ak = Ak.reshape(A.shape)
    Ak = np.moveaxis(Ak, 1, 0)

    return Ak, Ul, Ur


3. Compute the truncation error from sides _L_ and _R_.

$$
\varepsilon_L = \sum_{i > \chi} \vert S_{i,:,:,:} \vert^2
$$

$$
\varepsilon_R = \sum_{j > \chi} \vert S_{:,j,:,:} \vert^2
$$


In [5]:
def error_left(S: ArrayLike, max_bond: int) -> float:
    return np.sum(np.square(np.abs(S[max_bond:, :, :, :])))


def error_right(S: ArrayLike, max_bond: int) -> float:
    return np.sum(np.square(np.abs(S[:, max_bond:, :, :])))


4. Create a projector $U$ from the side that better approximates $M$, and normalize.
   - Normalizing against $\sqrt{\max(S)}$ seems to work.
   - $\sqrt{}$ is due to the fact that the projector is applied twice.
   - If max bond dimension $\chi$ is surpassed, truncate.

$$
\hat{U}^{(n)} = \begin{cases}
U^L / \sqrt{\max{S}} & \varepsilon_1 < \varepsilon_2 \\
U^R / \sqrt{\max{S}} & \text{otherwise}
\end{cases}
$$


5. Project $M$ with $\hat{U}$ to update $T$.

$$ T^{(n)}_{lrud} = \sum_{\alpha \beta} \hat{U}_{l,\alpha} M^{(n)}_{\alpha \beta u d} \hat{U}_{r,\beta} $$

In [6]:
def update_tensor(U: np.ndarray, M: np.ndarray, backend="numpy") -> np.ndarray:
    return oe.contract("il,ijud,jr->lrud", U, M, U, backend=backend)

In [7]:
def one_iteration(H: np.ndarray, T: np.ndarray, chi: int, backend="numpy") -> tuple[np.ndarray, np.ndarray]:
	# contract with environment
	Mh = contract_tensors(H, T, backend=backend)
	Mt = contract_tensors(T, T, backend=backend)

	# decompose tensor
	# S, Us = hotrg.hosvd(Mt)
	S, Ul, Ur = hosvd_sides(Mt)

	# compute projector
	epsl = error_left(S, chi)
	epsr = error_right(S, chi)
	U = Ul if epsl < epsr else Ur

	if U.shape[1] > chi:
		U = U[:, 0:chi]
	U /= np.sqrt(np.max(S))

	# update tensors
	H = update_tensor(U, Mh, backend=backend)
	T = update_tensor(U, Mt, backend=backend)

	return H, T

## Benchmark version

In [8]:
temperature = widgets.FloatSlider(min=2, max=5)
temperature

FloatSlider(value=2.0, max=5.0, min=2.0)

In [9]:
repeats = widgets.IntSlider(min=1, max=20)
repeats

IntSlider(value=1, max=20, min=1)

In [10]:
chi = widgets.IntSlider(value=8, min=8, max=24)
chi

IntSlider(value=8, max=24, min=8)

In [14]:
def trace(t):
	return oe.contract("iijj->", t)

## benchmark without transposition

### numpy

In [17]:
T = hotrg.ising.partition_tensor(temperature.value)
H = hotrg.ising.magnetization(temperature.value)
magnetization = [trace(H)/trace(T)]

with cProfile.Profile() as profile:
	for _ in tqdm(range(repeats.value)):
		for permutator in itertools.islice(itertools.cycle([(0,1,3,2), (2,3,0,1)]), 4):
				H, T = one_iteration(H, T, chi.value)
				H = H.transpose(permutator)
				T = T.transpose(permutator)
		print(f"magnetization = {trace(H)/trace(T)}")

  0%|          | 0/10 [00:00<?, ?it/s]

magnetization = -3.232063936613788e-16
magnetization = -1.7432541940774993e-15
magnetization = -4.043276130774083e-12
magnetization = 1.5715931606536523e-07
magnetization = 2.381222821784464e-06
magnetization = 3.785898048732858e-05
magnetization = 0.0006068945776025176
magnetization = 0.00971173897266511
magnetization = 0.1539079128211382
magnetization = 0.9039619793836275


In [18]:
profile.print_stats("tottime")

         131231 function calls (121304 primitive calls) in 10.204 seconds

   Ordered by: internal time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
       80    8.263    0.103    8.313    0.104 linalg.py:1477(svd)
 1020/780    0.967    0.001    9.374    0.012 {built-in method numpy.core._multiarray_umath.implement_array_function}
       40    0.133    0.003    0.149    0.004 4216494312.py:5(error_right)
      960    0.115    0.000    0.115    0.000 {method 'reshape' of 'numpy.ndarray' objects}
       40    0.112    0.003    8.464    0.212 3891133017.py:1(hosvd_sides)
       40    0.099    0.002    0.110    0.003 4216494312.py:1(error_left)
       59    0.073    0.001    0.073    0.001 socket.py:543(send)
      600    0.048    0.000    0.048    0.000 {method 'reduce' of 'numpy.ufunc' objects}
      240    0.046    0.000    0.046    0.000 {method 'astype' of 'numpy.ndarray' objects}
      180    0.023    0.000    0.099    0.001 contract.py:93(contract_path)
 

In [27]:
max_bond = chi.value
M = contract_tensors(T, T)
S, Ul, Ur = hosvd_sides(M)
error_left(S, max_bond)

1.5973618e-19

In [28]:
error_left(M, max_bond)

0.05336892

In [32]:
np.sum(np.square(np.abs(M)))

1.1956294

In [33]:
np.sum(np.square(np.abs(S)))

1.1956294

In [34]:
np.sum(np.square(np.abs(update_tensor(Ul, M))))

1.1956289

In [ ]:
np.sum(np.square(np.abs(update_tensor(Ul, M))))

1.1956289

In [35]:
np.sum(np.square(np.abs(update_tensor(Ur, M))))

1.195629